## Effective pedagogies for teaching data science

Mike Gelbart

UBC Jupyter Days 2020

In this session I will discuss a few tips for teaching data science, especially with Jupyter notebooks:

- [Problem-based learning](https://www.queensu.ca/ctl/teaching-support/instructional-strategies/problem-based-learning)
- [Live coding](https://en.wikipedia.org/wiki/Live_coding)
- My implementation of [think-pair-share](https://en.wikipedia.org/wiki/Think-pair-share).
- Real-time collaborative documents (perfect for online teaching!).
- (Time-permitting) Some thoughts on teaching in general.